<a href="https://colab.research.google.com/github/Cerino-rigo/EC3002C.602-2023/blob/main/manejo_datos_faltantes_categoricos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##El set de datos

Usaremos un set de datos que contiene la información del sexo, el peso (en Kg) y la altura de un grupo de 600 personas:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Importar librerías
import pandas as pd
import seaborn as sns



datos = pd.read_csv("/content/drive/MyDrive/Machine Learning/dataset_datos_faltantes_categoricos.csv")
datos

In [ ]:
# Distribución de estos datos para el peso
sns.histplot(data=datos, x='peso (kg)', hue='sexo');

In [ ]:
# Para la altura
sns.histplot(data=datos, x='altura (cm)', hue='sexo');

Podemos verificar si hay datos faltantes de varias formas.

La primera es con el método `info()`:

In [ ]:
datos.info()

Vemos que:

- En total debería haber 600 datos
- La columna `sexo` es categórica (`masculino` o `femenino`) y contiene 570 registros. Es decir, **faltan 30 registros**.
- Las columnas `peso (kg)` y `altura (cm)` son numéricas y están completas

Otra forma de ver la cantidad de datos faltantes es usando `isna()` y `sum()`:

In [ ]:
# Mostrar la cantidad de datos faltantes (marcados como NaN) en cada columna
datos.isna().sum()

O también podemos usar `value_counts()` aplicado directamente sobre la columna `sexo`:

In [ ]:
# Podemos verificar que la suma no es igual a 600
datos['sexo'].value_counts()

## Formas de manejar datos faltantes para variables categóricas

Cuando la variable "problemática" es categórica (como el caso de la columna `sexo`) podemos usar alguno de estos enfoques:

1. Eliminar las filas con los registros faltantes
2. Eliminar la columna "problemática"
3. Imputar con la categoría más frecuente
4. Imputar usando *Machine Learning* (**recomendado**)

Veamos cada una de estas técnicas junto con sus ventajas y desventajas:

###    Eliminar filas con los registros faltantes

Consiste simplemente en quitar la fila completa para cada registro faltante.

- Ventaja: ¡es el método más simple!
- Desventajas:
  - Si el dataset es "pequeño" la eliminación puede reducir significativamente su tamaño
  - Lo anterior puede dificultar tareas posteriores como, el uso de modelos de *Machine Learning* para generar predicciones

En el caso que nos interesa esta eliminación implica que por cada fila donde falta el dato del `sexo` también eliminaremos la información correspondiente a las columnas `peso (kg)` y `altura (cm)`.

La eliminación se puede hacer con el método `dropna()` de Pandas, que permite eliminar los registros que contienen datos *NaN*:

In [ ]:
df_filas = datos.dropna(axis=0) # Axis = 0: eliminar filas
df_filas.info()

Ya no tenemos datos faltantes pero hemos pasado de 600 a 570 registros en total.

### Eliminar columnas con los registros faltantes

Consiste simplemente en quitar la columna "problemática":

- Ventaja: ¡es el método más simple!
- Desventajas:
  - La eliminación de la columna "problema" puede dificultar tareas posteriores
  - ¿Vale la pena eliminar toda una columna cuando tan sólo faltan unos cuantos datos?

De nuevo, podemos usar el método `dropna()`:

In [ ]:
df_cols = datos.dropna(axis=1) # Axis = 1: eliminar columna(s)
df_cols.info()

Hemos preservado los 600 registros pero hemos eliminado una columna completa

### Imputar con la categoría más frecuente

Consiste en encontrar, en la columna "problema", la categoría (o nivel) que ocurre con mayor frecuencia y usarla para completar los datos faltantes.

Por ejemplo, veamos la categoría más común en la columna `sexo`:

In [ ]:
datos['sexo'].value_counts()

La categoría más común es `masculino` (288 datos en total). Así que en esta técnica de imputación usaríamos esta categoría para completar los datos faltantes.

- Ventaja: no se eliminan ni filas ni columnas.
- Desventaja: esta imputación puede generar sesgos

Para realizar esta imputación podemos usar el método `fillna()` de Pandas que nos permite "rellenar" los *NaN* con el valor que especifiquemos:

In [ ]:
# Generar una copia del DataFrame original
df_frec = datos.copy()

# Tomar la columna "sexo" y usar "fillna" para rellenar los valores
# faltantes con la categoría "masculino"
df_frec['sexo'] = df_frec['sexo'].fillna('masculino')

# Verificar que ya no hay valores faltantes
df_frec['sexo'].isna().sum()

**DataFrame.fillna(value=None, method=None)**

Rellena los valores NA/NaN utilizando el método especificado.

In [ ]:
df_frec

### Imputar usando *Machine Learning*

Es el método **más robusto y más recomendado**.

Consiste en construir un modelo de *Machine Learning* que tome las variables que están completas (por ejemplo `peso (kg)` y `altura (cm)`) y aprenda a predecir la variable incompleta (en este caso `sexo`):

![](https://drive.google.com/uc?export=view&id=1FPhsrSLu5TlHSoDtNSRgjMpxc7rD4qsa)


**Ventajas**
- Se preserva la cantidad de datos
- No se generan sesgos (siempre y cuando el modelo pueda ser construido correctamente)

**Desventajas**
- Se requieren suficientes datos para entrenar el modelo: no debe haber demasiados datos faltantes ni muy pocos datos de entrenamiento
- Dependiendo del set de datos no siempre resulta sencillo construir un modelo que genere predicciones adecuadas

**Procedimiento**
1. Crear el set de entrenamiento: registros que contienen datos completos
2. Crear el set de prueba: registros que contienen datos incompletos
3. Escoger y entrenar el modelo de *Machine Learning* con el set de entrenamiento
4. Predecir datos faltantes con el modelo entrenado y con el set de prueba
5. Incorporar los datos predichos en el dataset

Veamos en este ejemplo cómo implementar cada paso:

In [ ]:
# 1. Crear el set de entrenamiento

# 1.1. Extraer las filas que contienen datos completos
XY = datos.dropna().to_numpy()
XY.shape

In [ ]:
XY

In [ ]:
# 1.2. Set de entrenamiento
# x_train: columnas 1 y 2 ("peso (kg)" y "altura (cm)")
# y_train: columna 0 ("sexo")
x_train = XY[:,1:3]
y_train = XY[:,0]

print(x_train)
print(y_train)

In [ ]:
# 2. Crear el set de prueba: filas con datos incompletos
# y columnas "peso (kg)" y "altura (cm)"

filas = datos[~datos['sexo'].notna()].index # Filas incompletas
x_test = datos[['peso (kg)', 'altura (cm)']].iloc[filas].to_numpy()
x_test

El tercer paso es escoger y entrenar el modelo de Machine Learning con el set de entrenamiento.

Para este ejemplo podemos usar un sencillo modelo de Regresión Logística.

Pero antes de entrenarlo debemos pre-procesar los datos, pues las categorías a predecir (`masculino`, `femenino`) no pueden estar en formato de texto sino que deben estar en formato numérico (0 ó 1).

Para hacer esta conversión podemos usar `LabelEncoder` de *Scikit Learn*:

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(y_train)
le.classes_

In [ ]:
# Y ahora convertimos "y_train" a representación numérica
y_train = le.transform(y_train)
y_train

In [ ]:
# Después de construir el modelo, este generará predicciones
# numéricas (0, 1). Para obtener la categoría correspondiente
# usamos "inverse_transform". Por ejemplo:
le.inverse_transform([0,1,1,0])

Con los datos pre-procesados podemos construir y entrenar el modelo usando el módulo `LogisticRegression` de *Scikit Learn*:

In [ ]:
# Importar el módulo
from sklearn.linear_model import LogisticRegression

# Crear instancia del modelo
lr = LogisticRegression()

# Entrenarlo con "fit" y con los datos de entrenamiento
lr.fit(x_train,y_train)

¡Y con pocas líneas de código ya tenemos entrenado este modelo!

Veamos un ejemplo de predicción para entender cómo usarlo:

In [ ]:
entrada = [[69.,140.]]     # Supondremos un peso de 69 Kg y una altura de 120 cm
pred = lr.predict(entrada) # Generamos la predicción con predict
cat = le.inverse_transform(pred) # Y hacemos la transformación inversa

print(pred)
print(cat)

Ahora sólo debemos hacer lo mismo pero con el set de prueba (`x_test`) que, recordemos, es el que contiene los datos de peso y altura para las filas para las cuales desconocemos el sexo:

In [ ]:
# Predicciones sobre el set de prueba
preds = lr.predict(x_test)
print(preds)

# Transformaciones inversas
cats = le.inverse_transform(preds)
print(cats)

Lo único que queda es tomar las predicciones que se encuentran en la lista `cats` y realizar la imputación:

In [ ]:
# Ubicar las categorías predichas en las filas correspondientes
# de la columna "sexo" en el dataframe resultante
df_ml = datos.copy()
df_ml.iloc[filas,0]= cats # "sexo" es la columna 0
df_ml

In [ ]:
# Y verifiquemos que no hay datos faltantes
df_ml['sexo'].isna().sum()